In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
from holoviews import streams

In [ ]:
hv.extension('bokeh', 'matplotlib', logo=None)

In [ ]:
xvals = np.linspace(-5,5,256)
yvals = np.linspace(-5,5,256)
xs,ys = np.meshgrid(xvals, yvals)

def waves_image(alpha, beta):
    return np.sin(((ys/alpha)**alpha+beta)*xs)

In [ ]:
data_vars = {
    'i1': (['y', 'x'], waves_image(1., 1.)),
    'i2': (['y', 'x'], waves_image(1., 2.)),
    'i3': (['y', 'x'], waves_image(1., 3.)),
    'i4': (['y', 'x'], waves_image(1., 4.))
}
coords = {
    'x': (['x'], xvals, {'units':'a.u.'}), 
    'y': (['y'], yvals, {'units':'a.u'})
}
xrds = xr.Dataset(data_vars=data_vars, coords=coords)
xrds

In [ ]:
def bounds_to_text(bounds):
    xc = bounds[0] + (bounds[2] - bounds[0]) / 2
    yc = bounds[1] + (bounds[3] - bounds[1]) / 2
    txt = "x0: {:.2f}\ny0: {:.2f}\nx1: {:.2f}\ny1: {:.2f}".format(bounds[0], bounds[1], bounds[2], bounds[3])
    return hv.Text(xc, yc, txt, fontsize=10, halign='center', valign='center')

In [ ]:
def selection_callback(bounds):
    return (hv.Bounds(bounds) * bounds_to_text(bounds))

In [ ]:
%opts Image (cmap='viridis') [width=700 height=450 toolbar='above'  logz=False]
images = {en:hv.Image(ed) for en, ed in xrds.data_vars.items()}
hm = hv.HoloMap(images, kdims=['img'])
box = streams.BoundsXY(source=hm, bounds=(0,0,0,0))
bounds = hv.DynamicMap(selection_callback, streams=[box])
hm * bounds

In [ ]:
%opts Image [width=500 height=250]

class ImagesSelection(object):
    def __init__(self, imgs, kdims=[]):
        self.bounds = (0,0,0,0) 
        self.dynamic_map = dict()
        self.calls = list()
        for k, i in imgs.items():
            stream = streams.BoundsXY(source=i, bounds=(0,0,0,0))
            self.dynamic_map[k] = hv.DynamicMap(self.callback_factory(k), streams=[stream])
        self.bounds = hv.HoloMap(self.dynamic_map, kdims=imgs.kdims)
    
    def callback_factory(self, key):
        def callback(bounds):
            if self.bounds != bounds:
                self.bounds = bounds
                self.calls.append('{}: {}'.format(key, bounds))
                for k, dm in self.dynamic_map.items():
                    if k != key:
                        dm.event(bounds=bounds)
            return (hv.Bounds(self.bounds)* bounds_to_text(bounds))
        return callback
    
    def reset(self):
        self.bounds = (0,0,0,0) 
        for dm in self.dynamic_map.values():
            dm.event(bounds= self.bounds)
        
    def __mul__(self, hvobj):
        return hvobj * self.bounds
        
    def __rmul__(self, hvobj):
        return self.__mul__(hvobj)
    
images = hv.HoloMap({n:hv.Image(d) for n, d in xrds.data_vars.items()}, kdims=['img'])

selection = ImagesSelection(images)

hv.NdLayout(selection * images).cols(2)

In [ ]:
hv.NdLayout(hm).cols(2)

In [ ]:
hvds = hv.Dataset(xrds)
hvds

In [ ]:
%opts Curve [width=600 height=250 show_grid=True]
hvds.to(hv.Curve, ['x'], ['i1'])

In [ ]:
dv = {
    'S1': (['tx'], np.random.rand(1024)),
    'S2': (['tx'], np.random.rand(1024))
}
co = {
    'tx': (['tx'], np.linspace(-512, 512, 1024, True))
}
xrsp = xr.Dataset(data_vars=dv, coords=co)
xrsp

In [ ]:
hvsp = hv.Dataset(xrsp)
hvsp

In [ ]:
%opts Curve [width=1000 height=300  tools=['hover'] toolbar='above' shared_axes=True show_grid=True]
hvsp.to(hv.Curve, ['tx'], ['S1']) * hvsp.to(hv.Curve, ['tx'], ['S2'])